In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [5]:
from train_with_t5_loss import model_train_and_eval_t5
# from train_with_bart_loss import model_train_and_eval_bart

e:\anaconda\envs\coreferee_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
e:\anaconda\envs\coreferee_env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\MOO\.cache\huggingface\hub\models--goog

In [6]:
DATA_PATH = os.path.join("..", "data", "multinews")

In [7]:
scores = model_train_and_eval_t5(DATA_PATH)

Start training GNN. Parameters: hidden_size: 768, out_size: 768, attention_heads: 8
Acessing data path: ..\data\multinews\train.jsonl
Task runing on cpu
Start loading dataset...
Download finished. Creating embedding graph...
Finish graph creation
Dataset load successfully!
Setting finish. Start training epoch...


RuntimeError: DataLoader worker (pid(s) 5808, 14744, 13976, 120) exited unexpectedly

In [ ]:
scores_bart = model_train_and_eval_bart(DATA_PATH)

## gnn_train

In [2]:
from models.gnn_train import train_gnn, get_gnn_trained_embedding
import os

e:\anaconda\envs\coreferee_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
test_path = os.path.join("..","data", "multinews", "test.jsonl")

In [11]:
output_embeddings, node_sent_maps, summary_list = get_gnn_trained_embedding(
     evl_data_path = test_path,
     hidden_size=128,
     out_size=256,
     num_heads=3,
     batch_size=2
)

batch summary,  ['– The unemployment rate dropped to 8.2% last month, but the economy only added 120,000 jobs, when 203,000 new jobs had been predicted, according to today\'s jobs report. Reaction on the Wall Street Journal\'s MarketBeat Blog was swift: "Woah!!! Bad number." The unemployment rate, however, is better news; it had been expected to hold steady at 8.3%. But the AP notes that the dip is mostly due to more Americans giving up on seeking employment.', '– Shelly Sterling plans "eventually" to divorce her estranged husband Donald, she tells Barbara Walters at ABC News. As for her stake in the Los Angeles Clippers, she plans to keep it, the AP notes. Sterling says she would "absolutely" fight any NBA decision to force her to sell the team. The team is her "legacy" to her family, she says. "To be honest with you, I\'m wondering if a wife of one of the owners … said those racial slurs, would they oust the husband? Or would they leave the husband in?"']
summary list:  ['– The unemp

In [12]:
print(output_embeddings.shape)
print(len(node_sent_maps))
print(len(summary_list))

torch.Size([66, 256])
2
2


In [13]:
from models.t5 import get_t5_outputs

In [14]:
summary = get_t5_outputs(output_embeddings, node_sent_maps)

In [16]:
from utils.model_utils import rouge_eval

In [19]:
scs = rouge_eval("fake sum",summary)
print(scs)

{'rouge1': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.0, recall=0.0, fmeasure=0.0)}


In [20]:
scs['rouge1'].precision

0.0

In [47]:
train_gnn(
     file_path = test_path,
     hidden_size=128,
     out_size=256,
     num_heads=3,
     learning_rate=0.001,
     num_epochs=2,
     batch_size=2
)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch 1/2, Loss: 20.76664447784424
Epoch 2/2, Loss: 15.012077808380127
GNN Training Finish.


RelHetGraph(
  (lin_sent): Linear(in_features=768, out_features=768, bias=True)
  (lin_word): Linear(in_features=768, out_features=768, bias=True)
  (conv1): HeteroConv(num_relations=4)
  (conv2): HeteroConv(num_relations=4)
  (feat_drop): Dropout(p=0.2, inplace=False)
)

## optimized dataset loader test

In [2]:
from models.DatasetLoader import SummaryDataset, OptimizedDataset
from torch_geometric.loader import DataLoader as geo_DataLoader

e:\anaconda\envs\coreferee_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import os
file_path = os.path.join("..","data","multinews", "test.jsonl")

In [15]:
sum_data2 = OptimizedDataset(file_path)
dataloader2 = geo_DataLoader(sum_data2,batch_size=2,shuffle=False)

for batch in dataloader2:
     print(batch)

Loading data into shared memory...
Download finished. Creating embedding graph...
Finish graph creation, time cost:  35.0104 s.
len  2
get_item: id 0, data HeteroData(
  word={ x=[3, 768] },
  sentence={ x=[20, 768] },
  (sentence, similarity, sentence)={
    edge_index=[2, 6],
    edge_attr=[6, 1],
  },
  (sentence, pro_ant, sentence)={
    edge_index=[2, 6],
    edge_attr=[6, 1],
  },
  (sentence, has, word)={
    edge_index=[2, 8],
    edge_attr=[8, 1],
  },
  (word, in, sentence)={
    edge_index=[2, 8],
    edge_attr=[8, 1],
  }
)
get_item: id 1, data HeteroData(
  word={ x=[2, 768] },
  sentence={ x=[46, 768] },
  (sentence, similarity, sentence)={
    edge_index=[2, 26],
    edge_attr=[26, 1],
  },
  (sentence, pro_ant, sentence)={
    edge_index=[2, 66],
    edge_attr=[66, 1],
  },
  (sentence, has, word)={
    edge_index=[2, 7],
    edge_attr=[7, 1],
  },
  (word, in, sentence)={
    edge_index=[2, 7],
    edge_attr=[7, 1],
  }
)
HeteroDataBatch(
  word={
    x=[5, 768],
    b

In [5]:
### imporove process test
sum_data3 = OptimizedDataset(file_path)
dataloader3 = geo_DataLoader(sum_data3,batch_size=2,shuffle=False)

for batch in dataloader3:
     print(batch)

Loading data into shared memory...
Download finished. Creating embedding graph...


TypeError: unhashable type: 'list'